# 🦷 NHANES Periodontitis Prediction: Temporal Validation & Gradient Boosting Benchmark

**Author:** Francisco Teixeira Barbosa (Cisco @ Periospot)  
**Date:** November 2025  
**Project:** Replicating & Improving Bashir et al. (2022)

---

## 📄 Reference Paper

**Bashir NZ, Gill S, Tawse-Smith A, Torkzaban P, Graf D, Gary MT.**  
*Systematic comparison of machine learning algorithms to develop and validate predictive models for periodontitis.*  
**J Clin Periodontol.** 2022;49:958-969.

📁 **Paper Location:** `scientific_articles/J Clinic Periodontology - 2022 - Bashir...pdf`

---

## 🎯 Project Goals & Rationale

### The Problem
**Periodontitis** affects ~50% of US adults aged 30+, yet early prediction remains challenging.

**Bashir et al. (2022)** tested 10 ML algorithms and achieved:
- ✅ **Internal validation:** AUC > 0.95 (excellent)
- ❌ **External validation:** AUC ~0.50–0.60 (poor — no better than random!)

**Why the failure?** Cross-population validation (Taiwan ↔ US) is too strict—different healthcare systems, diagnostic criteria, and populations.

### Our Approach: Temporal Validation

Instead of validating across populations, we validate **across time within the same population**:

```
📅 TRAIN:      2011-2012 + 2013-2014  (~7,000 participants)
📅 VALIDATION: 2015-2016              (~3,500 participants)
📅 TEST:       2017-2018              (~3,500 participants)
```

**Why temporal?**
- ✅ Mimics real-world deployment: "Can a model trained on past data predict future patients?"
- ✅ Same population (US adults), same methodology (NHANES)
- ✅ More realistic than random shuffling
- ✅ Tests if model captures stable biological risk vs. temporal artifacts

### Methodological Improvements

1. **Modern Gradient Boosting:** XGBoost, CatBoost, LightGBM (NOT tested by Bashir)
2. **Hyperparameter Optimization:** Optuna Bayesian search
3. **Calibration:** Isotonic regression + decision curve analysis
4. **Interpretability:** SHAP feature importance
5. **Survey Weights:** Sensitivity analysis with NHANES complex sampling
6. **Reproducibility:** Versioned config, saved artifacts, git tracking

### Research Gap

From **Polizzi et al. (2024)** systematic review:  
> "None of the included articles used more powerful networks"

**Translation:** XGBoost, CatBoost, and LightGBM are **underutilized** in periodontitis prediction research.

**This study fills that gap.**

---

## 📊 Success Metrics

| Metric | Bashir Internal | Bashir External | **Our Target** |
|--------|----------------|-----------------|----------------|
| **AUC-ROC** | 0.95+ | 0.50–0.60 | **0.75–0.85** |
| **PR-AUC** | Not reported | Not reported | **0.60–0.75** |
| **Calibration** | Not reported | Not reported | **Brier < 0.20** |
| **Temporal Generalization** | N/A | Poor | **Better** |

**Realistic Expectation:** Even if we don't dramatically improve AUC, demonstrating that gradient boosting **doesn't magically solve external validation** is itself a **publishable finding** that advances the field.

---

## 🗺️ Notebook Roadmap

This notebook has **20 sections** organized into **5 phases**:

### Phase 1: Data Acquisition & Labeling (Sections 1–5)
1. Environment setup
2. Load configuration
3. Download NHANES data
4. Merge components
5. Apply CDC/AAP case definitions

### Phase 2: Feature Engineering & EDA (Sections 6–7)
6. Build 15 Bashir predictors
7. Exploratory analysis & drift detection

### Phase 3: Baseline Models (Sections 8–10)
8. Temporal train/val/test split
9. Preprocessing pipelines
10. Baseline models (LogReg, RandomForest)

### Phase 4: Gradient Boosting & Optimization (Sections 11–13)
11. XGBoost with Optuna
12. CatBoost with Optuna
13. LightGBM with Optuna

### Phase 5: Evaluation & Export (Sections 14–20)
14. Threshold selection on validation
15. Final test evaluation
16. Calibration & decision curves
17. SHAP interpretability
18. Survey weights sensitivity
19. Save artifacts & model card
20. Reproducibility log

---

## ⚠️ Important Notes Before Starting

1. **Read the Config First:** All parameters are in `configs/config.yaml`
2. **Implement TODOs Sequentially:** Each section builds on previous ones
3. **Test as You Go:** Run cells immediately to catch errors early
4. **Use Autocomplete:** Function signatures are provided—let your IDE help
5. **Don't Skip Section 5:** CDC/AAP classification is the most critical and brittle step
6. **Survey Weights:** For ML training, we use unweighted data (documented), but report weighted prevalence
7. **Freeze Threshold on Val:** Never touch test set until final evaluation

---

Let's begin! 🚀

In [2]:
"""
Section 1: Environment Setup & Imports
========================================
Set up the computational environment with all required libraries,
apply reproducibility measures, and configure Periospot plotting style.
"""

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    accuracy_score, recall_score, precision_score, f1_score,
    confusion_matrix, roc_curve, precision_recall_curve
)

import xgboost as xgb
import catboost as cb
import lightgbm as lgb

import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

import shap

import yaml
import json
from datetime import datetime

import sys
sys.path.insert(0, str(Path.cwd().parent / 'src'))
from ps_plot import set_style, get_palette, save_figure
from labels import label_periodontitis
from evaluation import compute_metrics, plot_roc_pr, select_threshold, plot_calibration_curve
from utils import set_seed, save_json, log_versions, save_model

RANDOM_SEED = 42
set_seed(RANDOM_SEED)

set_style()
palette = get_palette()
print("✅ Periospot color palette loaded:")
for name, hex_code in palette.items():
    print(f"   {name}: {hex_code}")

for dir_name in ['figures', 'models', 'results', 'artifacts', 'logs']:
    Path(dir_name).mkdir(exist_ok=True, parents=True)

print("\n📦 Package Versions:")
print(f"   pandas: {pd.__version__}")
print(f"   numpy: {np.__version__}")
print(f"   scikit-learn: {sklearn.__version__}")
print(f"   xgboost: {xgb.__version__}")
print(f"   catboost: {cb.__version__}")
print(f"   lightgbm: {lgb.__version__}")
print(f"   optuna: {optuna.__version__}")
print(f"   shap: {shap.__version__}")

print("✅ Section 1 Complete: Environment configured, seed set, Periospot style applied")

✅ Periospot color palette loaded:
   periospot_blue: #15365a
   mystic_blue: #003049
   periospot_red: #6c1410
   crimson_blaze: #a92a2a
   vanilla_cream: #f7f0da
   black: #000000
   white: #ffffff

📦 Package Versions:
   pandas: 2.3.2
   numpy: 2.3.5
   scikit-learn: 1.7.1
   xgboost: 3.1.1
   catboost: 1.2.8
   lightgbm: 4.6.0
   optuna: 4.6.0
   shap: 0.50.0
✅ Section 1 Complete: Environment configured, seed set, Periospot style applied


## 2️⃣ Load Configuration

**Load:** `configs/config.yaml`

**Contains:** NHANES cycles, temporal split, 15 predictors, CDC/AAP definitions, Optuna params, Periospot colors, survey weights

---

In [4]:
# TODO: Load config.yaml
with open("../configs/config.yaml") as f:
    config = yaml.safe_load(f)
TRAIN_CYCLES = config["temporal_split"]["train"]
VAL_CYCLES = config["temporal_split"]["validation"]
TEST_CYCLES = config["temporal_split"]["test"]
print(f"Train: {TRAIN_CYCLES}, Val: {VAL_CYCLES}, Test: {TEST_CYCLES}")
print("✅ Section 2: Config loaded")

Train: ['2011_2012', '2013_2014'], Val: ['2015_2016'], Test: ['2017_2018']
✅ Section 2: Config loaded


## 3️⃣ Download NHANES Data (XPT Files)

**Download** 4 cycles × 9 components = 36 XPT files from CDC

**Method:** `pd.read_sas(url)` → save as parquet

---

In [ ]:
# TODO: Loop cycles, download XPT files using pd.read_sas(url)
# for cycle in CYCLES:
#     cycle_dir = Path(f"data/raw/{cycle}")
#     cycle_dir.mkdir(parents=True, exist_ok=True)
#     for component, file_prefix in COMPONENTS.items():
#         url = f"{BASE_URL}/{cycle}/{file_prefix}_{suffix}.XPT"
#         df = pd.read_sas(url)
#         df.to_parquet(cycle_dir / f"{component}.parquet")
print("✅ Section 3: Data downloaded")

## 4️⃣ Merge Components on SEQN

**Join** all components by participant ID (SEQN)

**Filter:** Adults 30+

---

In [ ]:
# TODO: Merge all components on SEQN, filter age >= 30
# for cycle in CYCLES:
#     dfs = []
#     for comp in COMPONENTS:
#         df = pd.read_parquet(f"data/raw/{cycle}/{comp}.parquet")
#         dfs.append(df)
#     merged = dfs[0]
#     for df in dfs[1:]:
#         merged = merged.merge(df, on="SEQN", how="outer")
#     merged = merged[merged["RIDAGEYR"] >= 30]
#     merged.to_parquet(f"data/processed/{cycle}_merged.parquet")
print("✅ Section 4: Components merged")

## 5️⃣ Apply CDC/AAP Case Definitions

**Most Critical Section!**

**Implement:**
- Severe: CAL ≥6mm (≥2 different teeth) + PD ≥5mm (≥1 site)
- Moderate: CAL ≥4mm (≥2 teeth) OR PD ≥5mm (≥2 teeth)
- Mild: (CAL ≥3mm + PD ≥4mm on ≥2 teeth) OR PD ≥5mm (≥1 site)

**Use:** `src/labels.py` `label_periodontitis()`

---

In [ ]:
# TODO: Apply CDC/AAP classification using src/labels.py
# from labels import label_periodontitis
# for cycle in CYCLES:
#     df = pd.read_parquet(f"data/processed/{cycle}_merged.parquet")
#     df_labeled = label_periodontitis(df)
#     df_labeled.to_parquet(f"data/processed/{cycle}_labeled.parquet")
#     print(f"{cycle} prevalence: {df_labeled['has_periodontitis'].mean():.2%}")
print("✅ Section 5: CDC/AAP labels applied")

## 6️⃣ Build 15 Predictors

Extract Bashir predictors from NHANES variables

---

In [ ]:
# TODO: Build predictors
print("✅ Section 6: Predictors built")

## 7️⃣ Exploratory Analysis

Prevalence by cycle, missingness, drift

---

In [ ]:
# TODO: EDA plots
print("✅ Section 7: EDA complete")

## 8️⃣ Temporal Split

Train 2011-2014, Val 2015-2016, Test 2017-2018

---

In [ ]:
# TODO: Split by cycle
print("✅ Section 8: Temporal split done")

## 9️⃣ Preprocessing Pipelines

Imputation + scaling (fit on train only)

---

In [ ]:
# TODO: Build sklearn pipelines
print("✅ Section 9: Pipelines built")

## 🔟 Baseline Models

LogReg, RandomForest with 5-fold CV

---

In [ ]:
# TODO: Train baselines
print("✅ Section 10: Baselines trained")

## 1️⃣1️⃣ XGBoost + Optuna

Hyperparameter search, early stopping

---

In [ ]:
# TODO: Optuna tune XGBoost
print("✅ Section 11: XGBoost tuned")

## 1️⃣2️⃣ CatBoost + Optuna

Native categorical handling

---

In [ ]:
# TODO: Optuna tune CatBoost
print("✅ Section 12: CatBoost tuned")

## 1️⃣3️⃣ LightGBM + Optuna

Fast gradient boosting

---

In [ ]:
# TODO: Optuna tune LightGBM
print("✅ Section 13: LightGBM tuned")

## 1️⃣4️⃣ Threshold Selection

Choose policy (Youden, F1-max, Recall≥0.80), freeze on Val

---

In [ ]:
# TODO: Select threshold on Val
print("✅ Section 14: Threshold frozen")

## 1️⃣5️⃣ Final Test Evaluation

Apply frozen threshold, compute all metrics

---

In [ ]:
# TODO: Evaluate on Test
print("✅ Section 15: Test metrics computed")

## 1️⃣6️⃣ Calibration & Decision Curves

Isotonic/Platt scaling, net benefit

---

In [ ]:
# TODO: Calibration plots
print("✅ Section 16: Calibration done")

## 1️⃣7️⃣ SHAP Interpretability

Beeswarm + bar plots

---

In [ ]:
# TODO: SHAP analysis
print("✅ Section 17: SHAP complete")

## 1️⃣8️⃣ Survey Weights Sensitivity

Weighted prevalence with WTMEC2YR

---

In [ ]:
# TODO: Weighted stats
print("✅ Section 18: Survey weights applied")

## 1️⃣9️⃣ Save Artifacts

Export model, metrics, HF model card

---

In [ ]:
# TODO: Save all artifacts
print("✅ Section 19: Artifacts saved")

## 2️⃣0️⃣ Reproducibility Log

Package versions, git hash, system info

---

In [ ]:
# TODO: Log system info
print("✅ Section 20: Reproducibility logged")